In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv('cs-training-nonull.csv')

In [3]:
data = data.drop('id', axis = 1)

data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,isna_mi,isna_nod
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0,0,0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0,0,0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0,0,0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0,0,0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0,0,0


In [4]:
data = data.sample(frac = 1)

In [5]:
N_REC = data.shape[0]

N_TRAIN = int(N_REC * 0.8)
N_TEST = N_REC - N_TRAIN

train_df = data[:N_TRAIN]
test_df = data[N_TRAIN:]

In [6]:
xtrain_df = train_df.drop('SeriousDlqin2yrs', axis = 1)

x_train = xtrain_df.values

In [7]:
y_train = train_df['SeriousDlqin2yrs'].values

In [28]:
xtest_df = test_df.drop('SeriousDlqin2yrs', axis = 1)

x_test = xtest_df.values

y_test = test_df['SeriousDlqin2yrs'].values

In [29]:
param_dist = {'objective':'binary:logistic', 'n_estimators':50}

clf = xgb.XGBClassifier(use_label_encoder=False, **param_dist)

clf.fit(x_train, y_train,
        eval_set=[(x_train, y_train), (x_test, y_test)],
        eval_metric='auc',
        verbose=True)

evals_result = clf.evals_result()

[0]	validation_0-auc:0.84919	validation_1-auc:0.85217
[1]	validation_0-auc:0.85748	validation_1-auc:0.85868
[2]	validation_0-auc:0.85885	validation_1-auc:0.85910
[3]	validation_0-auc:0.85966	validation_1-auc:0.85939
[4]	validation_0-auc:0.86072	validation_1-auc:0.86017
[5]	validation_0-auc:0.86108	validation_1-auc:0.85972
[6]	validation_0-auc:0.86244	validation_1-auc:0.86000
[7]	validation_0-auc:0.86459	validation_1-auc:0.86183
[8]	validation_0-auc:0.86585	validation_1-auc:0.86258
[9]	validation_0-auc:0.86780	validation_1-auc:0.86255
[10]	validation_0-auc:0.86943	validation_1-auc:0.86267
[11]	validation_0-auc:0.87127	validation_1-auc:0.86270
[12]	validation_0-auc:0.87252	validation_1-auc:0.86296
[13]	validation_0-auc:0.87389	validation_1-auc:0.86341
[14]	validation_0-auc:0.87510	validation_1-auc:0.86369
[15]	validation_0-auc:0.87620	validation_1-auc:0.86425
[16]	validation_0-auc:0.87763	validation_1-auc:0.86468
[17]	validation_0-auc:0.87885	validation_1-auc:0.86489
[18]	validation_0-au

In [30]:
y_pred = clf.predict(x_test)

predictions = [round(value) for value in y_pred]

In [31]:
accuracy = accuracy_score(y_test, predictions)

print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.71%


In [32]:
# predictions on test set
orig_test = pd.read_csv('cs-test.csv')

orig_data = pd.read_csv('cs-training.csv')

# le statistiche per l normalizzazione le prendo dal train set
df_stats = orig_data.describe().transpose()

mean_mi = df_stats.loc['MonthlyIncome', 'mean']
mean_nod = df_stats.loc['NumberOfDependents', 'mean']

condition = (orig_test['MonthlyIncome'].isna())

orig_test['isna_mi'] = 0

orig_test.loc[condition, 'isna_mi'] = 1
orig_test.loc[condition, 'MonthlyIncome'] = mean_mi

In [22]:
# inpute 

condition = (orig_test['NumberOfDependents'].isna())

orig_test['isna_nod'] = 0

orig_test.loc[condition, 'isna_nod'] = 1
orig_test.loc[condition, 'NumberOfDependents'] = mean_nod

In [23]:
xorig_test = orig_test.drop('Unnamed: 0', axis = 1)

xorig_test = xorig_test.drop('SeriousDlqin2yrs', axis = 1)

x_test = xorig_test.values

In [24]:
y_pred = clf.predict_proba(x_test)

y_pred = y_pred[:, 1]

In [25]:
y_pred

array([0.07206833, 0.03773748, 0.01002266, ..., 0.00377197, 0.12527907,
       0.04222334], dtype=float32)

In [26]:
result_dict = {"Id": orig_test['Unnamed: 0'].values,
              'Probability': y_pred}

In [27]:
FILE_SUB = 'submission10.csv'

result_df = pd.DataFrame(result_dict)

result_df.to_csv(FILE_SUB, index=False, float_format='%.5f')